In [ ]:
import pandas as pd
from plotly import express as ex
from tqdm import trange

# MJ2383 Computer Lab 3

This lab aims to provide an inside view on what a supply curve is and how it can be generated while using OSeMOSYS. Reminder, a supply curve is a graphical representation of the law of supply. It slopes upward because the quantity supplied increases as price increases, with other things constant.

In this lab, we'll be using OSeMOSYS, but we'll be running it in the background using this Jupyter Notebook to control the input data, run the model and extract and visualise the results.

## Contents

- [Stage 1](#Stage-1:-A-super-simple-supply-curve) - In this section, we expore a very simple model with a two-stage supply curve for natural gas to develop our economic interpretation of OSeMOSYS
- [Stage 2]() - We add complexity, by increasing the number of steps in our supply curve by adding resources. We explore what difference this makes to the electricity price under different conditions.
- [Stage 3]() - We add an emissions penalty, imposing a tax upon CO2
- [Stage 4]() - We add renewable technologies to the electricity sector, whose marginal cost of generation is 0. However, this creates a demand for backup capacity. How does this affect the marginal price of electricity?

## Stage 1: A super simple supply curve

We start with the simplest supply cost curve you can imagine. In this simple OSeMOSYS model, there are two natural gas commodities (fuels) - imports of natural gas, and extraction. Both of these generate CO2, and feed a natural gas combined-cycle electricity generation plant. This produces secondary electricity `SEC_EL` which enters a transmission and distribution technology `TD` which produces final electricity `FEL`.

![](img/simple.svg)

(Note, this image is produced using the command `!otoole viz res SimpleEnergyModel/SimpleEnergyModel_Gas/datapackage.json img/simple.svg`)

---

First, let's have a think about the supply curve in this system.

Shown on the left of the image, there are two natural gas resources. `GasExtraction` has a maximum production capacity of 6 PJ/year at a variable cost of €8/PJ. However, `GasImport` has no upper bound, but a higher cost of €12/PJ.

Plotted, this looks rather uninspiring, but at least gives an impression of a supply curve for natural gas.


In [ ]:
gas_extraction = [["GasExtraction", x, 8] for x in range(7)]
gas_import = [["GasImport", x, 12] for x in range(6, 21)]
gas_supply_curve = pd.DataFrame(gas_extraction + gas_import, 
    columns=['name', 'supply', 'marginal_cost'])
fig = ex.line(gas_supply_curve, x='supply', y='marginal_cost', range_x=[0, 20], range_y=[0, 20], line_shape='vh')
fig.update_traces(mode="markers+lines")
fig.update_xaxes(showspikes=True)
fig.show()

In OSeMOSYS, supply must equal demand, and as the quantity demanded increases, supply increases accordingly. Given that our demand curve is a straight line (imagine a vertical line moving along the x-axis) the equilibrium point (where the lines cross) denotes the marginal cost of production and in this case is equal to the price.

Within most energy systems, there are multiple markets for different, related, energy commodities. We can use our simple OSeMOSYS model to begin to understand these different energy markets, and how they relate to one another.

Think about how our simple supply curve for natural gas interacts with the demand for electricity. 

In our simple example, we demand a specific quantity of electricity. The electricity is produced by the `NGCC` technology which requires almost 2 units of gas to produce 1 unit of electricity.

**Q. If the marginal cost of production of gas is €8/PJ, what is the marginal cost of production of electricity when 2 units of gas are needed to produce 1 unit of electricity?**

That's right, it should be around €16/PJ. Of course, it won't be exactly that, because in OSeMOSYS the marginal cost of production of electricity will take into account all the inputs into the production, including:
- capital cost of the extra capacity required
- fixed operating cost
- any costs associated with emissions

In addition, in OSeMOSYS, operating costs are discounted to the middle of the year, which is equivalent to a capital recovery factor of about 0.975 for the current year with a discount rate of 5%.

### Using OSeMOSYS to compute the equilibrium price

We'll now run OSeMOSYS from this Jupyter Notebook to compute the equilibrium price:

In [ ]:
def write_demand(value):
    """Write a ``value`` into the SpecifiedAnnualDemand file
    """
    demand = pd.DataFrame([['SIMPLICITY', 'FEL', 2014, value]], columns=["REGION","FUEL","YEAR","VALUE"])
    demand.to_csv("SimpleEnergyModel/SimpleEnergyModel_Gas/data/SpecifiedAnnualDemand.csv", index=False)

demand = 7
write_demand(demand)
!otoole convert datapackage datafile SimpleEnergyModel/SimpleEnergyModel_Gas/datapackage.json SimpleEnergyModel/SimpleEnergyModel_Gas/data_SimplicityModified_Gas.txt

In [ ]:
# Solve the model, writing the results into ./results
!glpsol -d SimpleEnergyModel/SimpleEnergyModel_Gas/data_SimplicityModified_Gas.txt -m osemosys.txt > osemosys.log

In the plot below, we can see the total production by fuel.  

**Q. Why is the production of `SEC_EL` higher than final electricity `FEL`?**

In [ ]:
production = pd.read_csv('results/ProductionByTechnologyAnnual.csv').groupby(by=['TECHNOLOGY', 'FUEL']).sum()
ex.bar(production.reset_index(), x='FUEL', y='VALUE', color='TECHNOLOGY')

In [ ]:
cost = pd.read_csv('results/ProductionDual.csv').set_index(['TIMESLICE', 'FUEL'])
cost['DUAL'] = cost['DUAL'] / (1.05**-0.5)
cost.loc[('WD', 'GAS'),'DUAL']

In [ ]:
fig = ex.line(gas_supply_curve, x='supply', y='marginal_cost', range_x=[0, 20], range_y=[0, 30], line_shape='vh')

gas_demand = production.groupby(by='FUEL').sum().loc['GAS', 'VALUE']

fig.add_shape( # add a vertical "demand" line
    type="line", line_color="salmon", line_width=3, opacity=1, line_dash="dot",
    x0=gas_demand, x1=gas_demand, xref="x", y0=0, y1=cost.loc[('WD', 'GAS'),'DUAL'])

fig.add_annotation( # add a text callout with arrow
    text="Marginal cost of gas", x=gas_demand, y=cost.loc[('WD', 'GAS'),'DUAL'], arrowhead=1, showarrow=True
)

fig.add_annotation( # add a text callout with arrow
    text="Marginal cost of electricity", x=gas_demand, y=cost.loc[('WD', 'FEL'),'DUAL'], arrowhead=1, showarrow=True
)

fig.show()

## Running the model to extract the marginal cost of production of electricity

In [ ]:
def run_model():
    results = []
    for dem in trange(1, 101, 5):
        observation = {}
        write_demand(dem)
        !otoole convert datapackage datafile SimpleEnergyModel/SimpleEnergyModel_Gas/datapackage.json SimpleEnergyModel/SimpleEnergyModel_Gas/data_SimplicityModified_Gas.txt
        !glpsol -d SimpleEnergyModel/SimpleEnergyModel_Gas/data_SimplicityModified_Gas.txt -m osemosys.txt > osemosys.log
        cost = pd.read_csv('results/ProductionDual.csv').set_index(['TIMESLICE', 'FUEL'])
        value = cost.loc[('WD', 'FEL'), 'DUAL']
        results.append({'value': value, 'param': 'FEL', 'demand': dem})
        value = cost.loc[('WD', 'GAS'), 'DUAL']
        results.append({'value': value, 'param': 'GAS', 'demand': dem})
    return results


In [ ]:
results = run_model()


In [ ]:
data = pd.DataFrame(results)
ex.line(data, x='demand', y='value', color='param', range_x=[0,100], range_y=[0,50])


In [ ]:
data = pd.DataFrame(results)
ex.line(data, x='demand', y='value', color='param', range_x=[0, 30], range_y=[0, 50])

# Stage 2: Adding Resources

# Stage 3: A tax on pollution

# Stage 4: Renewable electricity "there's no such thing as a free lunch"
